# 📘 Retrieval-Augmented Generation with Gemini & ChromaDB

This notebook demonstrates how to use Google's Gemini API along with ChromaDB to build a simple RAG (Retrieval-Augmented Generation) system.

## 🔧 Setup Instructions:
1. Install required libraries
2. Set your Gemini API key
3. Create vector embeddings of documents
4. Query using RAG

In [ ]:
# ✅ Step 1: Install necessary packages
!pip install -qU google-genai==1.7.0 chromadb==0.6.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.5 MB/s eta 0

In [ ]:
# ✅ Step 2: Set your Gemini API Key (replace with your actual key)
import os
os.environ["GOOGLE_API_KEY"] = "your-API-key"

In [ ]:
# ✅ Step 3: Initialize Gemini Client
from google import genai
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
# ✅ Step 4: List available embedding models
for model in client.models.list():
    if "embedContent" in model.supported_actions:
        print(model.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [ ]:
# ✅ Step 5: Prepare documents
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [ ]:
# ✅ Step 6: Define embedding function using Gemini
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
from google.genai import types

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

class GeminiEmbeddingFunction(EmbeddingFunction):
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        task_type = "retrieval_document" if self.document_mode else "retrieval_query"
        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(task_type=task_type),
        )
        return [e.values for e in response.embeddings]

In [ ]:
# ✅ Step 7: Create ChromaDB vector store
import chromadb

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name="googlecardb", embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])
print("DB size:", db.count())

DB size: 3


In [ ]:
# ✅ Step 8: Run a retrieval query
embed_fn.document_mode = False
query = "How do I play music using the touchscreen?"
result = db.query(query_texts=[query], n_results=1)
retrieved_docs = result["documents"][0]
print("Retrieved Passages:\n", retrieved_docs)

Retrieved Passages:
 ['Your Googlecar has a large touchscreen...']


In [ ]:
# ✅ Step 9: Use Gemini to generate an answer
prompt = f"""
You are a helpful bot. Answer the user's question using the passage below.
QUESTION: {query}
PASSAGE: {all_passages[0]}
"""

answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)

from IPython.display import Markdown
Markdown(answer.text)

The provided passage is "Y". This passage does not contain any information about playing music using the touchscreen. Therefore, I cannot answer your question.
